In [1]:
import numpy as np
import pandas as pd

In [2]:
a = np.random.rand(3,4)
a

array([[0.09475854, 0.60203748, 0.62575962, 0.89833279],
       [0.42411418, 0.4805884 , 0.73044355, 0.25234774],
       [0.12613908, 0.76269722, 0.38953068, 0.6558429 ]])